Import

In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy
from scipy.fftpack import fft, fftfreq, fftshift, ifft
from scipy import signal
from scipy.signal import butter, lfilter, filtfilt
from optcutfreq import optcutfreq

Read

In [7]:
import os, glob

In [8]:
# Define relative path to folder containing the text files

files_folder = "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/NonFallers_Processed_fc"
raw_files = []

# Create a dataframe list by using a list comprehension

raw_files = [pd.read_csv(file) for file in glob.glob(os.path.join(files_folder ,"*.csv"))]


Processing and save (4th order bidirectionnal Butterworth filter at 16hz with 0 phase shift)

In [9]:
Time= np.linspace(0,30,len(raw_files[0]))
def clean_df(df):
    '''
        Clean a df by removing the firs row
    '''
    return df.loc[:, ['COM_AP', 'COM_ML']]
# def residualanalysis_raw_data(column):
#     '''
#         Perform a residual analysis on each column to get the optimal cutoff frequency
#     '''
#     freq= np.mean(1/np.diff(Time))
#     fc_opt= optcutfreq(column.values, freq, show=False)
#     return fc_opt
def optcutfreq_store(df):
    '''
        Store each frequencies found in a list
    '''
    Opt_Freq_list= []   
    for i in df:
        Opt_Freq_list.append(10) #fc_opt #list of optimal cutoff frequencies for each columns
    return Opt_Freq_list
def Fourth_Order_bidirect_ButterW(column, fc_opt):
    '''
        4th Order Butterworth filter with 0 phase shift (low pass)(Hak et al. 2013)
    '''
    npasses= 2 # 2 pass (bidirectional)
    freq= np.mean(1/np.diff(Time))
    C = (2**(1/npasses) - 1)**0.25 #correction
    b, a = butter(4, (fc_opt/C)/(freq/4)) #4th order with correction the cutoff frequency for the number of passes in the filter
    filtered_column = filtfilt(b, a, column.values)  # filter with phase shift correction
    return filtered_column

def filtred_store(df):
    '''
        Store each filtred column  in a list
    '''
    filtred_col_list= [] 
    fc_opt_list= optcutfreq_store(df)
    Column_list= []   
    for i in df:
        Column_list.append(df[i]) #list of each column datas
    for column, fc_opt in zip(Column_list, fc_opt_list):
        filtred_col_list.append(Fourth_Order_bidirect_ButterW(column, fc_opt)) #list of optimal cutoff frequencies for each columns
    return filtred_col_list
def New_Data(df):
    '''
        Create a new dataframe with the filtered data
    '''
    names= list(df.columns)
    data = filtred_store(df)
    new_df = pd.DataFrame.from_dict(dict(zip(names, data)))  
    return new_df

CleanFiles= []
for i in raw_files:
    CleanFiles.append(clean_df(i))

ProcessedFiles= []
for i in CleanFiles:
    ProcessedFiles.append(New_Data(i))


In [10]:
raw_files[0]

,Unnamed: 0,COM_AP,COM_ML
0,499,-0.001361,0.000981
1,500,-0.001064,0.001054
2,501,-0.000767,0.001128
3,502,-0.000471,0.001203
4,503,-0.000177,0.001279
...,...,...,...
2996,3495,0.010000,0.011962
2997,3496,0.010205,0.011789
2998,3497,0.010406,0.011615
2999,3498,0.010606,0.011441


In [13]:
Time

array([0.000e+00, 1.000e-02, 2.000e-02, ..., 2.998e+01, 2.999e+01,
       3.000e+01])

In [11]:
# def save(lst_of_df):
#     for index in range(len(lst_of_df)):
#         lst_of_df[index].to_csv('processed_Fallers_[%s].csv' %index)

In [12]:
# save(ProcessedFiles)